In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,343 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,267 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,081 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jam

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

In [6]:
# Define the URL of the CSV file
from pyspark import SparkFiles
csv_url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
spark.sparkContext.addFile(csv_url)
charity_data_df = spark.read.csv(SparkFiles.get("charity_data.csv"), sep=",", header=True)

In [7]:
charity_data_df.show()

+--------+--------------------+----------------+----------------+--------------+------------+------------+------+-------------+----------------------+--------+-------------+
|     EIN|                NAME|APPLICATION_TYPE|     AFFILIATION|CLASSIFICATION|    USE_CASE|ORGANIZATION|STATUS|   INCOME_AMT|SPECIAL_CONSIDERATIONS| ASK_AMT|IS_SUCCESSFUL|
+--------+--------------------+----------------+----------------+--------------+------------+------------+------+-------------+----------------------+--------+-------------+
|10520599|BLUE KNIGHTS MOTO...|             T10|     Independent|         C1000|  ProductDev| Association|     1|            0|                     N|    5000|            1|
|10531628|AMERICAN CHESAPEA...|              T3|     Independent|         C2000|Preservation|Co-operative|     1|       1-9999|                     N|  108590|            1|
|10547893|ST CLOUD PROFESSI...|              T5|CompanySponsored|         C3000|  ProductDev| Association|     1|            0|   

In [13]:
updated_charity_data_df = charity_data_df.drop('EIN', 'NAME')

updated_charity_data_df.show()

+----------------+----------------+--------------+------------+------------+------+-------------+----------------------+--------+-------------+
|APPLICATION_TYPE|     AFFILIATION|CLASSIFICATION|    USE_CASE|ORGANIZATION|STATUS|   INCOME_AMT|SPECIAL_CONSIDERATIONS| ASK_AMT|IS_SUCCESSFUL|
+----------------+----------------+--------------+------------+------------+------+-------------+----------------------+--------+-------------+
|             T10|     Independent|         C1000|  ProductDev| Association|     1|            0|                     N|    5000|            1|
|              T3|     Independent|         C2000|Preservation|Co-operative|     1|       1-9999|                     N|  108590|            1|
|              T5|CompanySponsored|         C3000|  ProductDev| Association|     1|            0|                     N|    5000|            0|
|              T3|CompanySponsored|         C2000|Preservation|       Trust|     1|  10000-24999|                     N|    6692|       

In [17]:
updated_charity_data_df.printSchema()

root
 |-- APPLICATION_TYPE: string (nullable = true)
 |-- AFFILIATION: string (nullable = true)
 |-- CLASSIFICATION: string (nullable = true)
 |-- USE_CASE: string (nullable = true)
 |-- ORGANIZATION: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- INCOME_AMT: string (nullable = true)
 |-- SPECIAL_CONSIDERATIONS: string (nullable = true)
 |-- ASK_AMT: string (nullable = true)
 |-- IS_SUCCESSFUL: string (nullable = true)



In [19]:
updated_charity_data_df.createOrReplaceTempView('charity')

In [31]:
spark.sql("SELECT count(DISTINCT APPLICATION_TYPE) AS unique_count FROM charity").show()

+------------+
|unique_count|
+------------+
|          17|
+------------+



In [29]:
spark.sql("SELECT DISTINCT APPLICATION_TYPE FROM charity").show()

+----------------+
|APPLICATION_TYPE|
+----------------+
|             T13|
|              T6|
|              T9|
|              T3|
|              T7|
|             T14|
|             T12|
|              T5|
|             T19|
|             T10|
|              T2|
|             T25|
|              T8|
|             T17|
|             T29|
|             T15|
|              T4|
+----------------+



In [33]:
start_time = time.time()

spark.sql("SELECT DISTINCT CLASSIFICATION FROM charity").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------+
|CLASSIFICATION|
+--------------+
|         C2100|
|         C2570|
|         C1732|
|         C1600|
|         C1570|
|         C1240|
|         C3200|
|         C7210|
|         C2710|
|         C7200|
|         C1300|
|         C7000|
|         C1267|
|         C4500|
|         C1283|
|         C1237|
|         C1256|
|         C1280|
|         C1728|
|         C4100|
+--------------+
only showing top 20 rows

--- 1.4074532985687256 seconds ---


In [43]:
start_time = time.time()

spark.sql("SELECT DISTINCT IS_SUCCESSFUL FROM charity").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|IS_SUCCESSFUL|
+-------------+
|            0|
|            1|
+-------------+

--- 0.5811536312103271 seconds ---


In [44]:
spark.sql("cache table charity")

DataFrame[]

In [52]:
start_time = time.time()

spark.sql("SELECT DISTINCT IS_SUCCESSFUL FROM charity").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|IS_SUCCESSFUL|
+-------------+
|            0|
|            1|
+-------------+

--- 0.35675740242004395 seconds ---


In [53]:
spark.sql("uncache table charity")

DataFrame[]

In [54]:
# Verify that the table is no longer cached
if spark.catalog.isCached("charity"):
  print("a table is still cached")
else:
  print("all clear")

all clear
